In [1]:
# keras CNN predict air, between, ground or not jumping

In [2]:
!pip3 install tensorflow

Defaulting to user installation because normal site-packages is not writeable


In [4]:
import tensorflow as tf

from tensorflow.keras import datasets, layers, models
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cv2
import os
import pickle

from DataGeneratorFrames import DataGeneratorSkillBorders

In [5]:
video_border_labels_path = '../data/df_video_border_labels.pkl'
video_folder = '../videos/'

video_names = [
    '20240201_atelier_001.mp4',
    '20240201_atelier_002.mp4',
    '20240201_atelier_003.mp4',
    '20240201_atelier_004.mp4',
    '20240201_atelier_005.mp4',
    '20240209_atelier_006.mp4',
    '20240209_atelier_007.mp4',
    '20240209_atelier_008.mp4',
]

train_videos = [ video_folder + trainvid for trainvid in video_names]

In [6]:
def pickle_load_or_create(path, cols):
    if os.path.exists(path):
        with open(path, 'rb') as file:
            return pickle.load(file)
    else:
        return pd.DataFrame(columns=cols)

In [7]:
df_labels = pickle_load_or_create(video_border_labels_path, [])
df_labels

,path,frame,border
0,20240209_atelier_006.mp4,745,0
1,20240209_atelier_007.mp4,103,0
2,20240201_atelier_002.mp4,91,0
3,20240201_atelier_004.mp4,806,2
4,20240201_atelier_003.mp4,622,2
...,...,...,...
1170,20240209_atelier_006.mp4,99,0
1171,20240209_atelier_006.mp4,190,0
1172,20240201_atelier_005.mp4,387,2
1173,20240201_atelier_001.mp4,660,0


In [8]:
df_labels.loc[df_labels.border == 5, 'border'] = 3
df_labels.loc[df_labels.border == 9, 'border'] = 4

In [9]:
def get_random_frame(videos, grey=True, scale=1):
    """
    videos: array of video_paths
    df_video_border_labels: panda dataframe ['path', 'frame', 'borderlabel']
          0 : ground
          1 : heels of ground
          2 : air
          3 : Fault
          4 : no skipper or not jumping
    """
    path = videos[np.random.randint(0, len(videos)-1)]
    
    cap = cv2.VideoCapture(path)
    video_length = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    
    frame_nr = np.random.randint(0, video_length-1)
    
    cap.set(cv2.CAP_PROP_POS_FRAMES, frame_nr)
    res, frame = cap.read()
    frame = cv2.resize(frame, dsize=(0,0), fx=scale, fy=scale)
    # frame = cv2.cvtColor(frame, 7)
    cap.release()
    # cv2.destroyAllWindows()

    return path, frame_nr, frame

In [10]:
input_shape = get_random_frame(train_videos, grey=True, scale=0.4)[2].shape
input_shape

(768, 432, 3)

In [14]:
model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(64,64,3)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))


In [15]:
model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_3 (Conv2D)               │ (None, 62, 62, 32)     │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 31, 31, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_4 (Conv2D)               │ (None, 29, 29, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_3 (MaxPooling2D)  │ (None, 14, 14, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_5 (Conv2D)               │ (None, 12, 12, 64)     │        36,928 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 56,320 (220.00 KB)

 Trainable params: 56,320 (220.00 KB)

 Non-trainable params: 0 (0.00 B)

In [16]:
unique_labels = df_labels['border'].unique()
unique_labels

array([0, 2, 1, 4, 3])

In [17]:
model.add(layers.Flatten())
model.add(layers.Dense(16, activation='relu'))
model.add(layers.Dense(5))

In [18]:
model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_3 (Conv2D)               │ (None, 62, 62, 32)     │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 31, 31, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_4 (Conv2D)               │ (None, 29, 29, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_3 (MaxPooling2D)  │ (None, 14, 14, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_5 (Conv2D)               │ (None, 12, 12, 64)     │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 9216)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 16)             │       147,472 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 5)              │            85 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 203,877 (796.39 KB)

 Trainable params: 203,877 (796.39 KB)

 Non-trainable params: 0 (0.00 B)

In [21]:
# Parameters
params = {'dim': (64,64),
          'batch_size': 16,
          'n_classes': len(unique_labels),
          'n_channels': 3,
          'shuffle': True,
}

training_generator = DataGeneratorSkillBorders(df_labels, video_folder=video_folder, train=True, **params)
test_generator = DataGeneratorSkillBorders(df_labels, video_folder=video_folder, train=False, **params)

                          path  frame  border
609   20240209_atelier_007.mp4   1333       4
1052  20240201_atelier_001.mp4    373       0
728   20240201_atelier_001.mp4    517       2
785   20240201_atelier_001.mp4    726       4
919   20240201_atelier_005.mp4     59       4
...                        ...    ...     ...
718   20240201_atelier_005.mp4   1194       0
626   20240209_atelier_006.mp4    859       4
51    20240201_atelier_002.mp4    540       2
1125  20240209_atelier_006.mp4    645       0
1025  20240201_atelier_005.mp4   1289       0

[940 rows x 3 columns]
                          path  frame  border
288   20240209_atelier_007.mp4    622       1
864   20240209_atelier_006.mp4    437       0
392   20240201_atelier_002.mp4   1221       2
691   20240201_atelier_001.mp4    389       2
1158  20240201_atelier_001.mp4    253       2
...                        ...    ...     ...
203   20240201_atelier_003.mp4    433       1
1097  20240201_atelier_004.mp4   1305       2
1148  2024

In [22]:
%%time
X, y = training_generator.__getitem__(3)

CPU times: user 12.6 s, sys: 398 ms, total: 13 s
Wall time: 3.11 s


In [26]:
X.shape

(16, 64, 64, 3)

In [27]:
X

array([[[[0.34509804, 0.37647059, 0.41176471],
         [0.35294118, 0.38039216, 0.40392157],
         [0.35686275, 0.38823529, 0.42352941],
         ...,
         [0.31764706, 0.3372549 , 0.36078431],
         [0.32941176, 0.36078431, 0.37647059],
         [0.34509804, 0.36470588, 0.38823529]],

        [[0.36078431, 0.39215686, 0.42745098],
         [0.38039216, 0.4       , 0.42352941],
         [0.36470588, 0.38823529, 0.42352941],
         ...,
         [0.30980392, 0.34117647, 0.35686275],
         [0.32941176, 0.34901961, 0.37254902],
         [0.34901961, 0.37647059, 0.38823529]],

        [[0.34901961, 0.37254902, 0.40392157],
         [0.36078431, 0.39215686, 0.42745098],
         [0.36862745, 0.4       , 0.43529412],
         ...,
         [0.30196078, 0.31764706, 0.35294118],
         [0.32156863, 0.33333333, 0.35294118],
         [0.34117647, 0.35294118, 0.37254902]],

        ...,

        [[0.60392157, 0.59215686, 0.59607843],
         [0.60392157, 0.60392157, 0.59607843]

In [28]:
y

array([0, 4, 0, 4, 4, 2, 2, 0, 1, 0, 1, 0, 2, 0, 0, 2])

In [29]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

history = model.fit(training_generator, epochs=10, 
                    validation_data=test_generator)

Epoch 1/10


2024-06-22 18:56:27.234601: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:450] ShuffleDatasetV3:2: Filling up shuffle buffer (this may take a while): 4 of 8
2024-06-22 18:56:38.033190: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:450] ShuffleDatasetV3:2: Filling up shuffle buffer (this may take a while): 6 of 8


 1/58 ━━━━━━━━━━━━━━━━━━━━ 30:32 32s/step - accuracy: 0.1250 - loss: 1.6224

2024-06-22 18:56:46.493924: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:480] Shuffle buffer filled.


 3/58 ━━━━━━━━━━━━━━━━━━━━ 3:52 4s/step - accuracy: 0.1979 - loss: 1.5985

KeyboardInterrupt: 

In [ ]:
pd.DataFrame(history.history)

In [ ]:
with open('../models/last_cnn_model_history.pkl', 'wb') as handle:
    pickle.dump(history, handle)